In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import cross_validate


from sklearn import preprocessing



In [79]:
scaler = MinMaxScaler()



df= pd.read_csv("DFVariables_Election.csv", index_col = 0)
dftest = pd.read_csv("DFVariables_ElectionTEST.csv", index_col = 0)
# display(df)
X_train = df[["Social benefits","Homeprice","Income","Bankruptcy","Laborforce"]]
y_train = df["Electionresult"]
print(y_train)


X_train = scaler.fit_transform(X_train)
X_test =  dftest[["Social benefits","Homeprice","Income","Bankruptcy","Laborforce"]]
y_test = dftest["Electionresult"]
X_test = scaler.transform(X_test)


0     -0.339541
1     -0.567556
2     -0.522940
3     -0.507016
4     -0.602941
         ...   
351    0.199025
352    0.390655
353    0.185393
354    0.503244
355    0.219409
Name: Electionresult, Length: 1780, dtype: float64
[[0.36508568 0.79154104 0.96643634 0.3624417  0.55134364]
 [0.40962339 0.71988625 0.95967766 0.36012095 0.56833568]
 [0.40243114 0.80574885 0.96669837 0.3408575  0.52183975]
 ...
 [0.42552364 0.56518917 0.90875254 0.28312719 0.47334338]
 [0.44557527 0.55903363 0.90596671 0.28125689 0.43394495]
 [0.45267247 0.56314651 0.90796716 0.27453467 0.42780533]]


In [73]:
tscv = TimeSeriesSplit(n_splits = 4)


gsc = GridSearchCV(
        estimator=SVR(),
        param_grid=[{
            "kernel": ["rbf",'linear'],
            'C': [0.5, 1 ,5,10,15],
            'epsilon': [0.01,0.05, 0.1,0.15,0.2],
            'gamma': [ 1,1.5,2.0,2.5, 3.5]
        }],
        cv=tscv, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)

In [74]:
grid_result = gsc.fit(X_train, y_train)
best_params = grid_result.best_params_
print(best_params)


Fitting 4 folds for each of 250 candidates, totalling 1000 fits
{'C': 15, 'epsilon': 0.1, 'gamma': 3.5, 'kernel': 'rbf'}


In [76]:
regressor = SVR(kernel = "rbf",gamma =3.5,C = 15 , epsilon = 0.1)


print("Cross val r2: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "r2"))

print("Cross val r2 mean:")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "r2").mean())

print("\n")

print("Cross val RSME: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "neg_root_mean_squared_error"))

print("Cross val RSME mean: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "neg_root_mean_squared_error").mean())


regressor = SVR(kernel = "rbf",gamma = 9,C = 1.0 , epsilon = 0.1)
print("\n\n")
print(regressor)
print("Cross val r2: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "r2"))

print("Cross val r2 mean:")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "r2").mean())

print("\n")

print("Cross val RSME: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "neg_root_mean_squared_error"))

print("Cross val RSME mean: ")
print(cross_val_score(regressor, X_train,y_train, cv = tscv, scoring = "neg_root_mean_squared_error").mean())

Cross val r2: 
[-52.09279879 -12.39714151 -10.8823849  -12.51351834]
Cross val r2 mean:
-21.971460886360006


Cross val RSME: 
[-0.49181779 -0.30994114 -0.28886618 -0.56213088]
Cross val RSME mean: 
-0.4131889980081286



SVR(gamma=9)
Cross val r2: 
[-61.15355716 -11.0043999   -9.91668057 -13.17132931]
Cross val r2 mean:
-23.811491736929277


Cross val RSME: 
[-0.53213205 -0.2933887  -0.27687911 -0.57565002]
Cross val RSME mean: 
-0.41951246981058904


In [75]:
regressor = SVR(kernel = "rbf",gamma =3.5,C = 15 , epsilon = 0.1)

regressor.fit(X = X_train,y = y_train)
y_pred= regressor.predict(X_train)

print("R2 train:")
print(r2_score(y_train,y_pred))
print("RMSE train:")
print(np.sqrt(mean_squared_error(y_train,y_pred)))
y_pred = regressor.predict(X = X_test)
print("R2 test:")
print(r2_score(y_test,y_pred))
print("RMSE test:")
print(np.sqrt(mean_squared_error(y_test,y_pred)))



R2 train:
0.7469301083605842
RMSE train:
0.17215384419205434
R2 test:
-1.9284857284288486
RMSE test:
0.11179284942440552
